In [2]:
using Plots
using LaTeXStrings
using ControlSystemsBase

using ControlSafetyBench
using ControlTimingSafety

push!(LOAD_PATH, "../lib")
using Experiments

[ Info: Precompiling ControlTimingSafety [3a76c758-31f1-42cb-b697-12fe755e3b12]
[ Info: Precompiling Experiments [top-level]


In [3]:
sysc = benchmarks[:F1]

StateSpace{Continuous, Float64}
A = 
 0.0  6.5
 0.0  0.0
B = 
  0.0
 19.68503937007874
C = 
 1.0  0.0
D = 
 0.0

Continuous-time state-space model

In [23]:
H = 100
d_max = 1000
maxwindow = 6
params = (
    RC = (p = 23, d = 1.4, x0 = 100, n = 10),
    F1 = (p = 20, d = 1.2, x0 = 1,   n = 12),
    DC = (p = 23, d = 3.5, x0 = 100, n = 10),
    CS = (p = 27, d = 9.4, x0 = 100, n = 15),
    CC = (p = 28, d = 5.3, x0 = 10,  n = 20))
K = (
    RC = delay_lqr(benchmarks[:RC], 0.001*params[:RC][:p]),
    F1 = delay_lqr(benchmarks[:F1], 0.001*params[:F1][:p]),
    DC = delay_lqr(benchmarks[:DC], 0.001*params[:DC][:p]),
    CS = delay_lqr(benchmarks[:CS], 0.001*params[:CS][:p]),
    CC = delay_lqr(benchmarks[:CC], 0.001*params[:CC][:p]))

(RC = [0.16007906919762957 0.21425701514482584 0.022142131526773], F1 = [0.5829779235411586 0.9271753376618778 0.3501109341069689], DC = [0.002450925693814821 0.21008690565359417 0.009888275600972808], CS = [-0.060978904156291966 -0.029637457530448608 … -0.001946499703810083 0.19714862204522535], CC = [-0.17665541860563097 0.8461463805530567 1.0851122174471777 0.07462485824197998])

In [5]:
let
    nominal1 = nominal_trajectory_1(sysc, params[:F1][:x0], params[:F1][:p], H)
    nominal2 = nominal_trajectory_2(sysc, params[:F1][:x0], params[:F1][:p], H)
    slice_nominal(nominal2, params[:F1][:p]) ≈ nominal1
end

true

In [6]:
nominal = (
    RC = nominal_trajectory_2(benchmarks[:RC], params[:RC][:x0], params[:RC][:p], H),
    F1 = nominal_trajectory_2(benchmarks[:F1], params[:F1][:x0], params[:F1][:p], H),
    DC = nominal_trajectory_2(benchmarks[:DC], params[:DC][:x0], params[:DC][:p], H),
    CS = nominal_trajectory_2(benchmarks[:CS], params[:CS][:x0], params[:CS][:p], H),
    CC = nominal_trajectory_2(benchmarks[:CC], params[:CC][:x0], params[:CC][:p], H))

(RC = [100.0 100.0 0.0; 99.50147204362904 99.94996760575029 0.0; … ; -0.20545276106085425 15.901993024413743 -3.4221447456000185; -0.20543527442547238 15.889113976529561 -3.3590364680193625], F1 = [1.0 1.0 0.0; 1.0065 1.0 0.0; … ; 8.652486553805515e-7 -1.0535631128029171e-6 4.556859984924318e-7; 8.584296482869998e-7 -1.0445929159821999e-6 3.889085565468171e-7], DC = [100.0 100.0 0.0; 99.10438444336728 99.79821083016444 0.0; … ; 0.04715161003255562 0.3556994000380388 -0.08272282150209362; 0.047035933348003114 0.3548224909217961 -0.07817953814637621], CS = [100.0 100.0 … 100.0 0.0; 100.09999343412203 99.98040195534158 … 90.29016774501761 0.0; … ; 0.34324710066964254 39.42340443911915 … 5.9112603202036915 0.7128749720244894; 0.3826424579950375 39.367306813525815 … 5.888542184896456 0.6585405683797696], CC = [10.0 10.0 10.0 0.0; 10.010004980711097 10.009942129725069 9.88424519902869 0.0; … ; 0.5150580122629823 0.9532661809630534 0.12894333966375177 -1.428992417126292; 0.5160113409615763 0.

In [27]:
reff(s::Symbol) = synthesize_constraints(c2d(benchmarks[s], params[s][:p]*0.001),
    K[s], x_to_z_kill(benchmarks[s], params[s][:x0]), d_max, maxwindow, 
    params[s][:n], H, fullresults=true)[2]

reff (generic function with 1 method)

In [28]:
reff(:F1)

┌ Info: StateSpace{Discrete{Float64}, Float64}
│ A = 
│  1.0  0.12999999999999998
│  0.0  1.0
│ B = 
│  0.02559055118110236
│  0.39370078740157477
│ C = 
│  1.0  0.0
│ D = 
│  0.0
│ 
│ Sample Time: 0.02 (seconds)
│ Discrete-time state-space model
│   K =
│    1×3 Matrix{Float64}:
│     0.582978  0.927175  0.350111
│   z_0 =
│    3-element Vector{Float64}:
│     1.0
│     1.0
└     0.0
┌ Info: 1000
│   maxwindow = 6
│   n = 12
└   H = 100


6×6 Matrix{Float64}:
   0.0         Inf        Inf        Inf        Inf       Inf
   0.17864      0.0       Inf        Inf        Inf       Inf
   0.364089     0.17864    0.0       Inf        Inf       Inf
   4.00033      0.364089   0.17864    0.0       Inf       Inf
 468.581        2.73422    0.364089   0.17864    0.0      Inf
   1.89088e5  132.497      1.30196    0.364089   0.17864   0.0

In [17]:
pb(s::Symbol, newp::Real) = period_boosting(benchmarks[s], K[s], newp,
    x_to_z_kill(benchmarks[s], params[s][:x0]), d_max, maxwindow, params[s][:n], nominal[s])

pb (generic function with 1 method)

In [30]:
pb(:F1, 20)

┌ Info: StateSpace{Discrete{Float64}, Float64}
│ A = 
│  1.0  0.12999999999999998
│  0.0  1.0
│ B = 
│  0.02559055118110236
│  0.39370078740157477
│ C = 
│  1.0  0.0
│ D = 
│  0.0
│ 
│ Sample Time: 0.02 (seconds)
│ Discrete-time state-space model
│   K =
│    1×3 Matrix{Float64}:
│     0.582978  0.927175  0.350111
│   z_0 =
│    3-element Vector{Float64}:
│     1.0
│     1.0
└     0.0
┌ Info: 1000
│   maxwindow = 6
│   n = 12
└   H = 100


6×6 Matrix{Float64}:
   0.0         Inf        Inf        Inf        Inf       Inf
   0.17864      0.0       Inf        Inf        Inf       Inf
   0.364089     0.17864    0.0       Inf        Inf       Inf
   4.00033      0.364089   0.17864    0.0       Inf       Inf
 468.581        2.73422    0.364089   0.17864    0.0      Inf
   1.89088e5  132.497      1.30196    0.364089   0.17864   0.0